In [1]:
from pyspark import SparkContext


#创建SparkContext。
sc = SparkContext()


21/10/11 17:08:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
#读取文件并存储到RDD中。
rdd = sc.textFile('../Datasets/news/news_sentiment.csv')

In [3]:
#分布式统计文件的行数。
rdd.count()

4846

In [4]:
#查看前2行原始文件内容。
rdd.take(2)

['neutral,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing ."',
 'neutral,"Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said ."']

In [5]:
#用map构建标签的键-值对。
labels = rdd.map(lambda line: (line.split(',')[0], 1))

#用reduce对相同键的值进行求和。
label_counts = labels.reduceByKey(lambda a, b: a + b)

#查看标签的分布。
label_counts.take(5)

/opt/anaconda3/envs/python_dml/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/anaconda3/envs/python_dml/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[('negative', 604), ('positive', 1363), ('neutral', 2879)]

In [6]:
def extract_words(line):
    items = list()
    line = line.lower()
    words = line.split(',')[1]
    for word in words.split(' '):
        items.append((word, 1))
    return items
    
words = rdd.flatMap(extract_words)

word_counts = words.reduceByKey(lambda a, b: a + b)

#查看词频最高的10个单词。
word_counts.sortBy(lambda pair: pair[1], False).collect()[:10]

[('the', 3459),
 ('', 2590),
 ('.', 2068),
 ('of', 1908),
 ('in', 1596),
 ('to', 1561),
 ('and', 1144),
 ('a', 938),
 ('is', 688),
 ('for', 686)]